ConvLSTM->Conv network, trained on gridded forcing time-series and gridded geophysical data for all stations.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../..')
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
from datetime import datetime, timedelta
from sklearn import preprocessing
import netCDF4 as nc
import torch
from torch import nn, utils
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate, conv_lstm, datasets, utils
import torch.autograd as autograd
import pickle

time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
time_stamp

'20190917-195600'

In [2]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='../../log.out', mode='a')
chandler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - {} - %(message)s'.format(time_stamp))
fhandler.setFormatter(formatter)
chandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.addHandler(chandler)
logger.setLevel(logging.INFO)

In [3]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
num_devices = torch.cuda.device_count() if USE_CUDA else 0
logger.warning('cuda devices: {}'.format(list(torch.cuda.get_device_name(i) for i in range(num_devices))))
torch.manual_seed(0)
np.random.seed(0)

CUDA Available
2019-09-17 19:56:00,338 - 20190917-195600 - cuda devices: ['Tesla V100-SXM2-16GB']


In [4]:
seq_len = 8
seq_steps = 1
stateful_lstm = False
validation_fraction, val_start, val_end = None, None, None

if stateful_lstm:
    val_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(days=seq_len * seq_steps)  # first day for which to make a prediction in train set
    val_end = '2010-09-30'
    train_start = '2010-10-01'
    train_end = '2012-12-31'
else:
    validation_fraction = 0.1
    train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(days=seq_len * seq_steps)  # first day for which to make a prediction in train set
    train_end = '2012-12-31'
test_start = '2010-01-09'
test_end = '2014-12-31'

In [5]:
rdrs_vars = [4, 5]
agg = ['sum', 'minmax']
include_month = True
train_dataset = datasets.RdrsGridDataset(rdrs_vars, seq_len, seq_steps, train_start, train_end, aggregate_daily=agg, include_months=include_month, resample_rdrs=True)
if stateful_lstm:
    val_dataset = datasets.RdrsGridDataset(rdrs_vars, seq_len, seq_steps, val_start, val_end, conv_scalers=train_dataset.conv_scalers, aggregate_daily=agg, include_months=include_month, resample_rdrs=True)
test_dataset = datasets.RdrsGridDataset(rdrs_vars, seq_len, seq_steps, test_start, test_end, conv_scalers=train_dataset.conv_scalers, aggregate_daily=agg, include_months=include_month, resample_rdrs=True)

In [6]:
landcover_types = None
landcover, landcover_legend = load_data.load_landcover_reduced(values_to_use=landcover_types)
landcover = torch.from_numpy(landcover).float().to(device)

../../src/load_data.py:419: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  landcover_fullres = np.array(landcover_nc['Band1'][:])[::-1,:]


In [7]:
# Train model
num_epochs = 800
learning_rate = 2e-3
patience = 200
min_improvement = 0.01
best_loss_model = (-1, np.inf, None)

# Prepare model
batch_size = 32
num_convlstm_layers = 5
num_conv_layers = 5
convlstm_hidden_dims = [128,64,64,64,32]
conv_hidden_dims = [128,128,64,32]
convlstm_kernel_size = [(3,3)] * num_convlstm_layers
conv_kernel_size = [(3,3)] * num_conv_layers
conv_activation = nn.LeakyReLU
dropout = 0.2
weight_decay = 1e-6

model = conv_lstm.ConvLSTMGridWithGeophysicalInput((train_dataset.conv_height, train_dataset.conv_width), 
                                         train_dataset.n_conv_vars, landcover.shape[0], convlstm_hidden_dims, 
                                         conv_hidden_dims, convlstm_kernel_size, conv_kernel_size, 
                                         num_convlstm_layers, num_conv_layers, conv_activation, dropout=dropout,
                                                  feed_timesteps=seq_len).to(device)
if num_devices > 1:
    model = torch.nn.DataParallel(model, device_ids=list(range(num_devices)))
loss_fn = evaluate.NSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

writer = SummaryWriter(comment='ConvLSTM_withLandcover')
param_description = {'time_stamp': time_stamp, 'H_convlstm': convlstm_hidden_dims, 'H_conv': conv_hidden_dims, 'batch_size': batch_size, 'num_convlstm_layers': num_convlstm_layers, 'num_conv_layers': num_conv_layers, 'convlstm_kernel_size': convlstm_kernel_size, 'conv_kernel_size': conv_kernel_size, 'loss': loss_fn, 
                     'optimizer': optimizer, 'lr': learning_rate, 'patience': patience, 'min_improvement': min_improvement, 'stateful_lstm': stateful_lstm, 'dropout': dropout, 'landcover_shape': landcover.shape, 'conv_activation': conv_activation,
                     'num_epochs': num_epochs, 'seq_len': seq_len, 'seq_steps': seq_steps, 'train_start': train_start, 'train_end': train_end, 'weight_decay': weight_decay, 'validation_fraction': validation_fraction, 'landcover_types': landcover_types,
                     'test_start': test_start, 'test_end': test_end, 'n_conv_vars': train_dataset.n_conv_vars, 'model': str(model).replace('\n','').replace(' ', ''), 'val_start': val_start, 'val_end': val_end, 'aggregate_daily': agg,
                     'train len':len(train_dataset), 'conv_height': train_dataset.conv_height, 'conv_width': train_dataset.conv_width, 'test len': len(test_dataset), 'include_month': include_month}
writer.add_text('Parameter Description', str(param_description))
str(param_description)

"{'time_stamp': '20190917-195600', 'H_convlstm': [128, 64, 64, 64, 32], 'H_conv': [128, 128, 64, 32], 'batch_size': 32, 'num_convlstm_layers': 5, 'num_conv_layers': 5, 'convlstm_kernel_size': [(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)], 'conv_kernel_size': [(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)], 'loss': NSELoss(), 'optimizer': Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    eps: 1e-08\n    lr: 0.002\n    weight_decay: 1e-06\n), 'lr': 0.002, 'patience': 200, 'min_improvement': 0.01, 'stateful_lstm': False, 'dropout': 0.2, 'landcover_shape': torch.Size([19, 34, 39]), 'conv_activation': <class 'torch.nn.modules.activation.LeakyReLU'>, 'num_epochs': 800, 'seq_len': 8, 'seq_steps': 1, 'train_start': datetime.datetime(2010, 1, 9, 0, 0), 'train_end': '2012-12-31', 'weight_decay': 1e-06, 'validation_fraction': 0.1, 'landcover_types': None, 'test_start': '2010-01-09', 'test_end': '2014-12-31', 'n_conv_vars': 15, 'model': 'ConvLSTMGridWithGeophysicalInput((conv_lstm):C

In [8]:
if stateful_lstm:
    train_sampler = datasets.StatefulBatchSampler(train_dataset, batch_size)
    val_sampler = datasets.StatefulBatchSampler(val_dataset, batch_size)
    test_sampler = datasets.StatefulBatchSampler(test_dataset, batch_size)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_sampler, pin_memory=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_sampler=val_sampler, pin_memory=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_sampler, pin_memory=True)
else:
    val_indices = np.random.choice(len(train_dataset), size=int(validation_fraction * len(train_dataset)), replace=False)
    train_indices = list(i for i in range(len(train_dataset)) if i not in val_indices)
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=train_sampler, pin_memory=True, drop_last=False)
    val_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=val_sampler, pin_memory=True, drop_last=False)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=False)

In [9]:
torch.manual_seed(0)
np.random.seed(0)
for epoch in range(num_epochs):
    model.train()

    epoch_losses = torch.tensor(0.0)
    conv_hidden_states = None
    train_means = train_dataset.y_means.reshape(-1).to(device)
    for i, train_batch in enumerate(train_dataloader):
        y_train = train_batch['y'].reshape((train_batch['y'].shape[0],-1)).to(device, non_blocking=True)
        mask = train_batch['mask'].any(dim=0).reshape(-1).to(device, non_blocking=True)
        landcover_batch = landcover.repeat(y_train.shape[0],1,1,1).to(device)
        
        if not mask.any():
            print('Batch {} has no target values. skipping.'.format(i))
            continue
        if not stateful_lstm:
            conv_hidden_states = None
        
        y_pred, conv_hidden_states = model(train_batch['x_conv'].to(device), landcover_batch, hidden_state=conv_hidden_states)
        y_pred = y_pred.reshape((train_batch['y'].shape[0], -1))
        loss = loss_fn(y_pred[:,mask], y_train[:,mask], means=train_means[mask])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_losses += loss.detach()
        
    epoch_loss = (epoch_losses / len(train_dataloader)).item()
    print('Epoch', epoch, 'mean train loss:\t{}'.format(epoch_loss))
    writer.add_scalar('loss_nse', epoch_loss, epoch)
    
    # eval on validation split
    model.eval()
    val_losses = torch.tensor(0.0)
    conv_hidden_states = None
    for i, val_batch in enumerate(val_dataloader):
        y_val = val_batch['y'].reshape((val_batch['y'].shape[0],-1)).to(device, non_blocking=True)
        mask = val_batch['mask'].any(dim=0).reshape(-1).to(device, non_blocking=True)
        landcover_batch = landcover.repeat(y_val.shape[0],1,1,1).to(device)
        
        if not stateful_lstm:
            conv_hidden_states = None
        
        batch_pred, conv_hidden_states = model(val_batch['x_conv'].to(device), landcover_batch, hidden_state=conv_hidden_states)
        batch_pred = batch_pred.detach().reshape((val_batch['y'].shape[0], -1))
        val_losses += loss_fn(batch_pred[:,mask], y_val[:,mask], means=train_means[mask]).detach()
        
    val_nse = (val_losses / len(val_dataloader)).item()
    print('Epoch {} mean val loss:  \t{}'.format(epoch, val_nse))
    writer.add_scalar('loss_nse_val', val_nse, epoch)
    if val_nse < best_loss_model[1] - min_improvement:
        best_loss_model = (epoch, val_nse, model.state_dict())  # new best model
        load_data.pickle_model('ConvLSTM_withLandcover', model, 'allStations', time_stamp)
    elif epoch > best_loss_model[0] + patience:
        print('Patience exhausted in epoch {}. Best val-loss was {}'.format(epoch, best_loss_model[1]))
        break
    
print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
model.load_state_dict(best_loss_model[2])
load_data.pickle_model('ConvLSTM_withLandcover', model, 'allStations', time_stamp)

Epoch 0 mean train loss:	1.2413785457611084
Epoch 0 mean val loss:  	1.175148367881775
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/ConvLSTM_withLandcover_allStations_20190917-195600.pkl
Epoch 1 mean train loss:	1.1512451171875
Epoch 1 mean val loss:  	1.2575082778930664
Epoch 2 mean train loss:	1.1322845220565796
Epoch 2 mean val loss:  	1.1118885278701782
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/ConvLSTM_withLandcover_allStations_20190917-195600.pkl
Epoch 3 mean train loss:	1.079087734222412
Epoch 3 mean val loss:  	0.991722822189331
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/ConvLSTM_withLandcover_allStations_20190917-195600.pkl
Epoch 4 mean train loss:	1.0566246509552002
Epoch 4 mean val loss:  	1.0608199834823608
Epoch 5 mean train loss:	1.0324658155441284
Epoch 5 mean val loss:  	1.0895887613296509
Epoch 6 mean train loss:	1.0372166633605957
Epoch 6 mean val loss:  	1.1443407535552979
Epoch 7 mean train loss:	1.0089805126190186
Epoc

In [10]:
logger.warning('predicting')
model.eval()

predictions = []
conv_hidden_states = None
for i, test_batch in enumerate(test_dataloader):
    if not stateful_lstm:
        conv_hidden_states = None
        
    landcover_batch = landcover.repeat(test_batch['y'].shape[0],1,1,1).to(device)
    pred, conv_hidden_states = model(test_batch['x_conv'].to(device), landcover_batch, hidden_state=conv_hidden_states)
    predictions.append(pred.detach())

predictions = torch.cat(predictions).cpu()

if stateful_lstm:
    # reorder time series
    pred_indices = np.array(list(test_sampler.__iter__())).reshape(-1)
    predictions = predictions[pred_indices.argsort()]

2019-09-17 22:05:42,377 - 20190917-195600 - predicting


In [11]:
actuals = test_dataset.data_runoff.copy()
if len(actuals['date'].unique()) != len(predictions):
    print('Warning: length of prediction {} and actuals {} does not match.'.format(len(predictions), len(actuals['date'].unique())))

nse_list = []
mse_list = []
predictions_df = pd.DataFrame(columns=actuals.columns)
for station in actuals['station'].unique():
    row, col = test_dataset.station_to_index[station]
    
    act = actuals[actuals['station'] == station].set_index('date')['runoff']
    if predictions.shape[0] < act.shape[0]:
        print('Warning: length of prediction {} and actuals {} does not match for station {}. Ignoring excess actuals.'.format(len(predictions), len(act), station))
        act = act.iloc[:predictions.shape[0]]
    if predictions.shape[0] > act.shape[0]:
        print('Warning: length of prediction {} and actuals {} does not match for station {}. Ignoring excess predictions. (Likely this station had nan ground truth values)'.format(len(predictions), len(act), station))
    pred = pd.DataFrame({'runoff': predictions[:,row,col]}, index=test_dataset.dates)
    pred = pred.loc[act.index]
    pred['station'] = station
    predictions_df = predictions_df.append(pred.reset_index(), sort=True)
    
    nse, mse = evaluate.evaluate_daily(station, pred['runoff'], act, writer=writer)
    nse_list.append(nse)
    mse_list.append(mse)
    
    print(station, '\tNSE:', nse, '\tMSE:', mse, '(clipped to 0)')

print('Median NSE (clipped to 0)', np.median(nse_list), '/ Min', np.min(nse_list), '/ Max', np.max(nse_list))
print('Median MSE (clipped to 0)', np.median(mse_list), '/ Min', np.min(mse_list), '/ Max', np.max(mse_list))
writer.add_scalar('nse_median', np.median(nse_list))

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


04213500 	NSE: 0.6073308320090619 	MSE: 349.2416600205949 (clipped to 0)
04209000 	NSE: 0.7250408109193266 	MSE: 127.80030598313002 (clipped to 0)
04164000 	NSE: 0.6175834044458346 	MSE: 61.3972139934129 (clipped to 0)
04159900 	NSE: 0.6796054055930566 	MSE: 10.838652893415135 (clipped to 0)
04200500 	NSE: 0.6842920291396086 	MSE: 322.57207076449527 (clipped to 0)
04160600 	NSE: 0.7254136067958085 	MSE: 7.638148920954077 (clipped to 0)
04208504 	NSE: 0.7416143735732573 	MSE: 373.473895069709 (clipped to 0)
04199500 	NSE: 0.6872273845429939 	MSE: 149.32258687033894 (clipped to 0)
02GG002 	NSE: 0.6696420109544556 	MSE: 64.02647268364039 (clipped to 0)
02GA038 	NSE: 0.5059420121066316 	MSE: 56.44547163759325 (clipped to 0)
04196800 	NSE: 0.7017040750371196 	MSE: 90.32437920061685 (clipped to 0)
04212100 	NSE: 0.7741199438167246 	MSE: 461.31631766518103 (clipped to 0)
04174500 	NSE: 0.6885964422193083 	MSE: 48.94569453903733 (clipped to 0)
02GG003 	NSE: 0.6967999149389107 	MSE: 140.0390261

In [12]:
writer.close()

In [13]:
save_df = pd.merge(predictions_df.rename({'runoff': 'prediction'}, axis=1), actuals.rename({'runoff': 'actual'}, axis=1), 
                   on=['date', 'station'])[['date', 'station', 'prediction', 'actual']]
load_data.pickle_results('ConvLSTM_withLandcover', save_df, time_stamp)

'ConvLSTM_withLandcover_20190917-195600.pkl'

In [14]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20190917-220603'